# Lesson 5：Python `os` 库详解

`os` 模块提供了访问操作系统功能的统一接口，帮助 Python 程序在不同平台上处理文件、目录、环境变量以及进程信息

## 1. 模块概览：为何使用 `os`

- 为不同操作系统提供一致的 API，编写跨平台脚本更加容易。
- 与操作系统交互：路径拼接、目录遍历、权限检查、环境变量管理等。
- 与底层系统调用保持接近，适合需要精细控制的场景。

## 2. 运行环境与基础信息

In [1]:
# 读取基础环境信息，便于根据平台调整逻辑
import os

print(f"当前平台标识 os.name: {os.name}")               # posix / nt / java
print(f"CPU 核心数 os.cpu_count(): {os.cpu_count()}")      # 返回可用 CPU 数量

# os.uname 在 POSIX 系统可用，Windows 使用 platform 模块补充
if hasattr(os, "uname"):
    uname = os.uname()
    print("os.uname():", uname)
else:
    print("os.uname() 在当前平台不可用，可使用 platform 模块获取信息")

print(f"当前进程号 os.getpid(): {os.getpid()}")
print(f"父进程号 os.getppid(): {os.getppid()}")


当前平台标识 os.name: posix
CPU 核心数 os.cpu_count(): 8
os.uname(): posix.uname_result(sysname='Darwin', nodename='shenyufeideMacBook-Air.local', release='25.0.0', version='Darwin Kernel Version 25.0.0: Wed Sep 17 21:41:23 PDT 2025; root:xnu-12377.1.9~141/RELEASE_ARM64_T8122', machine='arm64')
当前进程号 os.getpid(): 26097
父进程号 os.getppid(): 75478


## 3. 环境变量操作

In [2]:
# os.environ 暴露了一个可变映射，读取与设置环境变量
import os

# 读取常用环境变量，如果不存在则返回默认值
python_path = os.environ.get("PYTHONPATH", "<未设置>")
print(f"PYTHONPATH: {python_path}")

# 临时设置一个变量，供子进程或后续代码使用
os.environ["DEMO_FLAG"] = "enabled"
print(f"DEMO_FLAG: {os.environ['DEMO_FLAG']}")

# setdefault 适合在缺失时写入默认值
value = os.environ.setdefault("APP_MODE", "development")
print(f"APP_MODE: {value}")


PYTHONPATH: /Users/shenyufei/workspace:/Users/shenyufei/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev:/Users/shenyufei/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/jupyter_debug
DEMO_FLAG: enabled
APP_MODE: development


## 4. 路径操作与 `os.path`

In [3]:
# os.path 子模块负责路径拼接、解析与属性判断
import os

base_dir = os.path.expanduser("~")       # 展开用户主目录
project_dir = os.path.join(base_dir, "workspace", "pytest")

print(f"主目录: {base_dir}")
print(f"项目目录（拼接）: {project_dir}")
print(f"是否绝对路径: {os.path.isabs(project_dir)}")
print(f"路径拆分: {os.path.split(project_dir)}")
print(f"文件扩展名拆分: {os.path.splitext('report.csv')}")
print(f"规范化路径: {os.path.normpath('src/../src/utils')}")


主目录: /Users/shenyufei
项目目录（拼接）: /Users/shenyufei/workspace/pytest
是否绝对路径: True
路径拆分: ('/Users/shenyufei/workspace', 'pytest')
文件扩展名拆分: ('report', '.csv')
规范化路径: src/utils


## 5. 工作目录管理

In [5]:
# 通过 getcwd/chdir 管理当前工作目录，执行后记得切回原目录
import os

original_cwd = os.getcwd()                # 保存原目录，方便恢复
print(f"原始工作目录: {original_cwd}")

try:
    # 切换到用户主目录，展示 chdir 的效果
    target = os.path.expanduser("~")
    os.chdir(target)
    print(f"切换后目录: {os.getcwd()}")
finally:
    # 确保恢复，以免影响其他单元执行
    os.chdir(original_cwd)
    print(f"已恢复目录: {os.getcwd()}")


原始工作目录: /Users/shenyufei/workspace/pytest
切换后目录: /Users/shenyufei
已恢复目录: /Users/shenyufei/workspace/pytest


## 6. 文件与目录的创建、遍历、删除

In [6]:
# 使用临时目录安全地演示文件系统操作
import os
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    print(f"临时工作区: {tmpdir}")

    # 创建嵌套目录结构
    nested_dir = os.path.join(tmpdir, "data", "raw")
    os.makedirs(nested_dir, exist_ok=True)     # exist_ok 避免重复创建报错

    # 创建示例文件
    file_path = os.path.join(nested_dir, "demo.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("迭代器课程 -> os 模块示例")

    # 列出当前层级的内容
    print("os.listdir(tmpdir):", os.listdir(tmpdir))

    # 使用 scandir 获取更丰富的条目信息
    with os.scandir(tmpdir) as it:
        for entry in it:
            print(f"scandir -> name={entry.name}, is_dir={entry.is_dir()}")

    # 删除文件与空目录
    os.remove(file_path)
    os.rmdir(nested_dir)
    print("删除后内容:", os.listdir(os.path.join(tmpdir, "data")))


临时工作区: /var/folders/wx/ys8_0hqn7pb7rk1y_nv3fvbh0000gn/T/tmpk9r_yn6v
os.listdir(tmpdir): ['data']
scandir -> name=data, is_dir=True
删除后内容: []


## 7. 深度遍历：`os.walk`

In [5]:
# os.walk 递归遍历层级目录，返回 (root, dirs, files) 元组
import os
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    # 构造目录树: tmpdir/project/{src,tests}/...
    for relative in [
        ("project", "src", "main.py"),
        ("project", "src", "utils.py"),
        ("project", "tests", "test_main.py"),
    ]:
        dir_path = os.path.join(tmpdir, *relative[:-1])
        os.makedirs(dir_path, exist_ok=True)
        open(os.path.join(dir_path, relative[-1]), "w").close()

    for root, dirs, files in os.walk(os.path.join(tmpdir, "project")):
        print(f"当前位置: {root}")
        print(f"  子目录: {dirs}")
        print(f"  文件: {files}")


当前位置: /var/folders/wx/ys8_0hqn7pb7rk1y_nv3fvbh0000gn/T/tmpvy0oecdw/project
  子目录: ['tests', 'src']
  文件: []
当前位置: /var/folders/wx/ys8_0hqn7pb7rk1y_nv3fvbh0000gn/T/tmpvy0oecdw/project/tests
  子目录: []
  文件: ['test_main.py']
当前位置: /var/folders/wx/ys8_0hqn7pb7rk1y_nv3fvbh0000gn/T/tmpvy0oecdw/project/src
  子目录: []
  文件: ['utils.py', 'main.py']


## 8. 文件元数据与权限

In [6]:
# os.stat 返回底层 inode 信息，包含大小、权限、时间戳等
import os
import stat
import tempfile

with tempfile.NamedTemporaryFile(delete=False) as tmp:
    tmp.write(b"os module metadata demo")
    temp_name = tmp.name

try:
    info = os.stat(temp_name)
    print(f"文件大小（字节）: {info.st_size}")
    print(f"最后访问时间: {info.st_atime}")
    print(f"最后修改时间: {info.st_mtime}")
    print(f"权限位（八进制）: {oct(info.st_mode & 0o777)}")

    # 使用 stat 模块判断类型
    print("是否常规文件:", stat.S_ISREG(info.st_mode))
    print("是否目录:", stat.S_ISDIR(info.st_mode))
finally:
    os.remove(temp_name)


文件大小（字节）: 23
最后访问时间: 1761735386.5484166
最后修改时间: 1761735386.548708
权限位（八进制）: 0o600
是否常规文件: True
是否目录: False


## 9. 进程与文件描述符

In [ ]:
# os 模块还能操作底层进程与文件描述符，这里演示只读信息
import os
import tempfile

print(f"当前进程 ID: {os.getpid()}")
print(f"父进程 ID: {os.getppid()}")

# 使用临时文件获取文件描述符，避免依赖 __file__
with tempfile.NamedTemporaryFile() as tmp:
    fd = tmp.fileno()
    print(f"临时文件的描述符: {fd}")
    print(f"使用 os.isatty(fd) 判断是否关联终端: {os.isatty(fd)}")


## 10. 与 `pathlib` 的协作

In [3]:
# pathlib 提供面向对象的路径操作，与 os 可相互转换
from pathlib import Path
import os

path_obj = Path.cwd() / "data" / "sample.csv"
print(f"Path 对象: {path_obj}")
print(f"转换为字符串后可与 os.path 配合: {os.path.dirname(str(path_obj))}")

# pathlib.Path 也能直接调用 os 风格的方法
print(f"是否存在: {path_obj.exists()}")
print(f"父目录列表: {list(path_obj.parents)}")


Path 对象: /Users/shenyufei/workspace/pytest/data/sample.csv
转换为字符串后可与 os.path 配合: /Users/shenyufei/workspace/pytest/data
是否存在: False
父目录列表: [PosixPath('/Users/shenyufei/workspace/pytest/data'), PosixPath('/Users/shenyufei/workspace/pytest'), PosixPath('/Users/shenyufei/workspace'), PosixPath('/Users/shenyufei'), PosixPath('/Users'), PosixPath('/')]


## 11. 权限管理：`chmod` 与 `umask`

- `os.chmod` 修改文件权限位，常与八进制表示法配合。
- `os.umask` 控制新建文件的默认权限掩码，可暂时设置后恢复。

In [7]:
# 演示 chmod 与 umask，用临时文件防止影响真实文件
import os
import tempfile

with tempfile.NamedTemporaryFile(delete=False) as tmp:
    path = tmp.name

try:
    initial_mode = oct(os.stat(path).st_mode & 0o777)
    print(f"初始权限: {initial_mode}")

    os.chmod(path, 0o640)
    print(f"修改后权限: {oct(os.stat(path).st_mode & 0o777)}")

    if hasattr(os, "umask"):
        previous = os.umask(0)
        os.umask(previous)  # 立即恢复，避免影响后续代码
        print(f"当前进程的 umask: {oct(previous)}")
    else:
        print("当前平台未提供 os.umask")
finally:
    os.remove(path)


初始权限: 0o600
修改后权限: 0o640
当前进程的 umask: 0o22


## 12. 链接与文件引用

- `os.link` 创建硬链接，多个名称指向同一个 inode。
- `os.symlink` 创建符号链接，类似快捷方式；可通过 `os.readlink` 查看实际目标。
- 某些平台限制该功能，应先检查接口是否存在。

In [8]:
# 通过临时目录演示符号链接与硬链接
import os
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    origin = os.path.join(tmpdir, "origin.txt")
    with open(origin, "w", encoding="utf-8") as f:
        f.write("os link demo")

    if hasattr(os, "symlink"):
        symlink_path = os.path.join(tmpdir, "origin_symlink.txt")
        os.symlink(origin, symlink_path)
        print(f"符号链接创建成功? {os.path.islink(symlink_path)}")
        print(f"符号链接指向: {os.readlink(symlink_path)}")
    else:
        print("当前平台不支持符号链接")

    if hasattr(os, "link"):
        hardlink_path = os.path.join(tmpdir, "origin_hard.txt")
        os.link(origin, hardlink_path)
        same_inode = os.stat(origin).st_ino == os.stat(hardlink_path).st_ino
        print(f"硬链接共享 inode? {same_inode}")
        print(f"硬链接数量: {os.stat(origin).st_nlink}")
    else:
        print("当前平台不支持硬链接")


符号链接创建成功? True
符号链接指向: /var/folders/wx/ys8_0hqn7pb7rk1y_nv3fvbh0000gn/T/tmp31nhkul9/origin.txt
硬链接共享 inode? True
硬链接数量: 2


## 13. 低层文件描述符操作

- `os.open`/`os.read`/`os.write` 暴露 C 语言风格的系统调用接口。
- `os.lseek` 调整文件指针，`os.fdopen` 将描述符包装成 Python 文件对象。
- 用于需要精准控制缓冲或与底层库协作的场景。

In [ ]:
# 演示使用低层文件描述符并与高层对象互转
import os
import tempfile

with tempfile.NamedTemporaryFile(delete=False) as tmp:
    path = tmp.name

try:
    fd = os.open(path, os.O_RDWR | os.O_CREAT | os.O_TRUNC, 0o600)
    try:
        os.write(fd, "低层写入".encode("utf-8"))
        os.lseek(fd, 0, os.SEEK_SET)
        raw = os.read(fd, 1024)
        print(f"os.read -> {raw.decode('utf-8').strip()}")

        os.lseek(fd, 0, os.SEEK_END)
        with os.fdopen(fd, "a+", encoding="utf-8") as handle:
            handle.write("通过 fdopen 追加")
            handle.seek(0)
            print("fdopen ->", handle.read().splitlines())
    finally:
        # fd 已被 fdopen 关闭，无需再次 os.close
        pass
finally:
    os.remove(path)


## 14. 管道与进程间通信

- `os.pipe` 建立匿名管道，返回一对文件描述符（读端、写端）。
- 写端关闭后，读端会接收 EOF，可结合 `select`、`subprocess` 实现更复杂通信。

In [ ]:
# 使用管道在同一进程内模拟 producer/consumer 通信
import os

read_fd, write_fd = os.pipe()
try:
    message = "使用 os.pipe 传递数据".encode("utf-8")
    os.write(write_fd, message)
    os.close(write_fd)  # 关闭写端以便读端检测到 EOF

    received = os.read(read_fd, 1024)
    print(received.decode("utf-8"))
finally:
    os.close(read_fd)


## 15. 小结与最佳实践

- `os` 模块贴近操作系统，为 Python 提供跨平台的系统调用接口。
- 环境变量、路径、目录、元数据是最常用的基础能力，建议配合 `tempfile` 做安全演示。
- 通过 `chmod`/`umask`、链接、底层文件描述符以及 `os.pipe` 可深入了解系统资源管理。
- 根据需求在 `os`、`pathlib`、`subprocess` 间切换，确保代码既可维护又具备必要的底层控制力。